In [1]:
import os
# Prevent CUDA from using GPU as it does not work well on my pc
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

# Set Constants of the model
BATCH_SIZE = 64
SHUFFLE_BUFFER_SIZE = 100

In [2]:
# Helper functions
import numpy as np

# Breaks down a list of integer values into a one-hot like format
def one_hot_training(np_array):
    transformed_list = []
    for arr in np_array:
        new_arr = np.zeros(10)
        new_arr[int(arr)] = 1
        transformed_list.append(new_arr)
    return np.array(transformed_list)

# This translates the highest value from the one-hot encoding into the correct sign name
def one_hot_translator(np_array):
    labels_names = ['Stop','Yield','Red Light','Green Light','Roundabout','Right Turn Only',
                'Do Not Enter','Crosswalk','Handicap Parking','No Parking']
    return labels_names[np.argmax(np_array)]

# This translates an entire array of one-hot encoded sign predictions
def translate_all(np_array):
    translated_values = []
    for i in np_array:
        translated_values.append(one_hot_translator(i))
    return np.array(translated_values)

In [3]:
# First import the data
import tensorflow as tf
data_train = np.load('data_train.npy').transpose()
labels_train = np.load('labels_train.npy')
data_train = np.array([i.reshape(300,300,3) for i in data_train])
data_train = np.array(tf.cast(tf.image.resize(data_train,(75,75)), np.uint8))

In [4]:
# Process the data so that it is in the expected form for the InceptionV3 model
import tensorflow as tf
processed = tf.keras.applications.inception_v3.preprocess_input(data_train, data_format=None)

# Break down data into training and test sets
from sklearn.model_selection import train_test_split
x_train, x_test, t_train, t_test = train_test_split(processed, one_hot_training(labels_train), test_size=0.20, random_state=1)

In [5]:
# Augment data to reduce overfitting

from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(horizontal_flip=True,
                                   vertical_flip=True,
                                   rotation_range=90,
                                   brightness_range=(0.25, 0.75))

train_generator = train_datagen.flow(
    x_train,
    y = t_train,
    batch_size=BATCH_SIZE)

In [6]:
# Import the InceptionV3 Model

from tensorflow.keras.applications.inception_v3 import InceptionV3
inception = InceptionV3(input_shape=(75,75,3),
                       include_top=False,
                       weights='imagenet')

# Set layers to false to prevent overwriting the existing model
for layer in inception.layers:
    layer.trainable = False

# Create output layers that will be trained
from tensorflow.keras.optimizers import SGD
x = tf.keras.layers.Flatten()(inception.output)
x = tf.keras.layers.Dense(1024, activation="relu")(x)
x = tf.keras.layers.Dropout(0.15)(x)
x = tf.keras.layers.Dense(10, activation='softmax')(x)

# Create Optimizer
Adam = tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False, name='Adam') 
Nadam = tf.keras.optimizers.Nadam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07,name='Nadam') 
SGD = SGD(learning_rate=0.01, nesterov=False)
optimizer = SGD

# Finalize and compile the model
model = tf.keras.Model(inception.input, outputs = x)
model.compile(optimizer = optimizer,
             loss = 'categorical_crossentropy',
             metrics = ['acc'])

In [ ]:
# Fit the model to the dataset
callbacks = tf.keras.callbacks.Callback()
history = model.fit(train_generator, epochs=10, batch_size=BATCH_SIZE)
model.save("100_epoch.h5")

Epoch 1/10
78/78 [==============================] - 28s 252ms/step - loss: 1.3830 - acc: 0.5494
Epoch 2/10
78/78 [==============================] - 20s 251ms/step - loss: 0.8815 - acc: 0.7331
Epoch 3/10
78/78 [==============================] - 20s 251ms/step - loss: 0.7790 - acc: 0.7603
Epoch 4/10
27/78 [=========>....................] - ETA: 12s - loss: 0.7260 - acc: 0.7760

In [ ]:
# Plot the progression of the acccuracy through the epochs
import matplotlib.pyplot as plt
plt.plot(history.history['acc'])
plt.title('model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
# Demo to randomly pick a sign and demonstrate that it is predicted correctly

from random import randint
test_image = randint(0,len(data_train))
test = np.expand_dims(data_train[test_image], axis=0)
test = tf.keras.applications.inception_v3.preprocess_input(
    test, data_format=None
)
print(one_hot_translator(model.predict(test)))
plt.imshow(data_train[test_image])

In [ ]:
# Calculate the predictions for the test values
predictions = model.predict(x_test)

In [ ]:
evaluation = model.evaluate(x_test, t_test)
print("Test run accuracy is {}".format(evaluation[-1]))

In [ ]:
# Create a Confusion Matrix to show the weakness in the model
predicted_values = translate_all(predictions)
real_values = translate_all(t_test)
from sklearn.metrics import confusion_matrix
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
cfm = confusion_matrix(real_values, predicted_values)
disp = ConfusionMatrixDisplay(confusion_matrix=cfm, display_labels=['Stop','Yield','Red Light','Green Light','Roundabout','Right Turn Only','Do Not Enter','Crosswalk','Handicap Parking','No Parking'])
disp.plot()
plt.show()

In [ ]:
# Test Run Data

# Testing optimizers
# NADAM=                                   times = [275,272,286], loss after 3 runs = [19.0231,0.5492,0.4377], accuracy after 3 runs = [0.6372,0.8364,0.8650]
# ADAM=                                    times = [190,186,190], loss after 3 runs = [16.5848,0.5596,0.4106], accuracy after 3 runs = [0.6245,0.8307,0.8763]
# SGD(.001, Nesterov=No, Momentum = No),   times = [182,180,181], loss after 3 runs = [1.1017, 0.5662,0.4399], accuracy after 3 runs = [0.6269,0.8238,0.8660]
# SGD(.01, Nesterov=No, Momentum = No),    times = [180,180,179], loss after 3 runs = [2.6334,0.5418,0.3895], accuracy after 3 runs = [0.5662,0.8210,0.8854]
# SGD(.01, Nesterov=Yes, Momentum = 0.25), times = [183,183,183], loss after 3 runs = [2.8777,0.5341,0.4083], accuracy after 3 runs = [0.5361,0.8234,0.8741]
# SGD(.01, Nesterov=Yes, Momentum = 0.5),  times = [189,183,183], loss after 3 runs = [3.3616,0.4973,0.4002], accuracy after 3 runs = [0.5696,0.8351,0.8763]
# SGD(.01, Nesterov=Yes, Momentum = 0.75), times = [185,185,184], loss after 3 runs = [5.2539,0.5209,0.4359], accuracy after 3 runs = [0.5337,0.8212,0.8606]

In [ ]:
from random import randint
test_image = randint(0,len(x_train))
plt.imshow(x_train[test_image])

In [ ]:
print(train_generator.next)